In [36]:
import nltk
import parse_corpus
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import numpy as np

import lda

# Split into sections

In [6]:
filename = "preprocessed_shakespeare.txt"

sections = parse_corpus.process(filename)

In [10]:
# with open("/Users/markmartinez/Desktop/shakespeare.txt","r") as F:
#     allText = F.read().lower()

# Remove stopwords, tokenize, stem (takes about 30 seconds)

In [25]:
stops = stopwords.words("english")
stopdict = dict((s,None) for s in stops) # Sets are really terrible in Python

ps = PorterStemmer()

clean_sections = []

# NOTE(tfs): I am using tokenization now. Mainly it really cleans up punctuation and handles contractions well
for section in sections:
    secwords = section.split()
    nonstops = [w.lower() for w in secwords if not w in stopdict]
    tokens = nltk.word_tokenize(" ".join(nonstops))
    clean_sections.append(" ".join([ps.stem(t) for t in tokens if t.isalnum()])) # Note this does not preserve structure,
                                              #      but all words are now present in the section string
print clean_sections[0][:500]

rousillon the count palac enter bertram countess of rousillon helena lafeu black countess in deliv son me i buri second husband bertram and i go madam weep father death anew i must attend majesti command i ward evermor subject lafeu you shall find king husband madam you sir father he gener time good must necess hold virtu you whose worthi would stir want rather lack abund countess what hope majesti amend lafeu he hath abandon physician madam whose practic hath persecut time hope find advantag pr


In [49]:
# stops = set(stopwords.words("english"))
# words = allText.split()
# meaningful_words = [w for w in words if not w in stops]
# cleanedUp =  " ".join( meaningful_words )

In [9]:
# cleanedUp[0:500]

In [53]:
#now split on 'scene'
# scenes = [x.strip() for x in cleanedUp.split("scene")]

In [38]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word")

data = vectorizer.fit_transform(clean_sections)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = data.toarray()

In [39]:
# print sum(train_data_features[300])

In [40]:
# vocab = vectorizer.get_feature_names()
# print(len(vocab))

In [41]:
# import numpy as np

# # Sum up the counts of each vocabulary word
# dist = np.sum(data, axis=0)

# # For each, print the vocabulary word and the number of times it 
# # appears in the training set
# for tag, count in zip(vocab, dist):
#     print(count, tag)

# Run LDA (takes a bit of time)

In [46]:
# Settings
num_topics = 20
num_iter = 500
n_top_words = 10

In [47]:
model = lda.LDA(n_topics=num_topics, n_iter=num_iter)
model.fit(data)

topic_word = model.topic_word_

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 643
INFO:lda:vocab_size: 13409
INFO:lda:n_words: 466303
INFO:lda:n_topics: 20
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -5204287
INFO:lda:<10> log likelihood: -4232255
INFO:lda:<20> log likelihood: -4061967
INFO:lda:<30> log likelihood: -3989114
INFO:lda:<40> log likelihood: -3942837
INFO:lda:<50> log likelihood: -3904528
INFO:lda:<60> log likelihood: -3876797
INFO:lda:<70> log likelihood: -3851616
INFO:lda:<80> log likelihood: -3827616
INFO:lda:<90> log likelihood: -3803176
INFO:lda:<100> log likelihood: -3781590
INFO:lda:<110> log likelihood: -3766195
INFO:lda:<120> log likelihood: -3752205
INFO:lda:<130> log likelihood: -3736597
INFO:lda:<140> log likelihood: -3725179
INFO:lda:<150> log likelihood: -3715526
INFO:lda:<160> log likelihood: -3703483
INFO:lda:<170> log likelihood: -3695325
INFO:lda:<180> log likelihood: -3687659
INFO:lda:<190> log likelihood: -3682436
INFO:lda:<200> log likelihood: -3673757
INFO:lda:<210> log likelihood: -3669990
INFO:lda:<

Topic 0: soldier enter franc us the shall lord talbot fight french
Topic 1: you to lord the love that shall one it thi
Topic 2: macbeth prospero macduff sebastian the ariel antonio stephano ladi banquo
Topic 3: come what well say he good man is would enter
Topic 4: brutu caesar citizen cassiu coriolanu meneniu rome you first marciu
Topic 5: helena parol love bertram and lafeu demetriu lysand hermia bottom
Topic 6: and the to that may for but heart like hand
Topic 7: troilu hector pandaru cressida achil thersit ajax ulyss agamemnon diomed
Topic 8: king franc berown princess of armado john costard bastard katharin
Topic 9: of iago othello cassio syracus desdemona dromio antipholu imogen ephesu
Topic 10: titu and portia luciu marcu bassanio rome antonio shylock aaron
Topic 11: valentin and proteu petruchio sir love julia speed tranio lucentio
Topic 12: lord timon second servant apemantu posthumu iachimo third senat men
Topic 13: duke angelo isabella claudio provost brother sir lucio pedro